In [1]:
from modules.lstm_encoder import LSTM_encoder
from modules.lstm_decoder import LSTM_decoder
from keras.regularizers import l2
import os
import numpy as np

Using TensorFlow backend.


## Encoding

In [2]:
encoder = LSTM_encoder(l2, './features/lstm/2_steps/', './convnet_weights/lstm/base/weights.19-1.13.hdf5')

In [3]:
features_dev = encoder.encode('dev')

26/26 [==============================] - 13s 518ms/step


In [4]:
features_dev.shape

(3313, 2, 2048)

## Classes

In [17]:
indices = np.load('./features/lstm/2_steps/indices_dev.npy')
true_classes = np.load('./features/res_net/classes_dev_make-model.npy')[indices]
true_classnames = np.load('./features/res_net/classnames_dev_make-model.npy')[indices]
true_filenames = np.load('./features/res_net/filenames_dev.npy')[indices]

In [22]:
true_classes

array([ 54,  89, 164, ..., 136,  14,   3])

In [21]:
true_classnames

array(['Chevrolet Silverado_1500', 'Ford F-450', 'Volvo 240', ...,
       'Mercedes-Benz 300-Class', 'Audi S6', 'Acura TL'],
      dtype='<U33')

In [19]:
true_filenames

array(['Chevrolet Silverado_1500 Regular_Cab 2012/02808.jpg',
       'Ford F-450 Crew_Cab 2012/05187.jpg',
       'Volvo 240 Sedan 1993/04113.jpg', ...,
       'Mercedes-Benz 300-Class Convertible 1993/07798.jpg',
       'Audi S6 Sedan 2011/00602.jpg', 'Acura TL Sedan 2012/03169.jpg'],
      dtype='<U58')

## Predictions

In [35]:
def map_predictions_to_classnames(predictions, true_classes, classnames):
    classes, indices = np.unique(true_classes, return_index=True)
    
    class_to_index = {}
    for cl, index in zip(classes, indices):
        class_to_index[cl] = index
    
    predicted_classnames = list(map(lambda cl: classnames[class_to_index[cl]], predictions))
    return np.array(predicted_classnames)

In [12]:
decoder = LSTM_decoder(features_dev, true_classes, weights_path='./convnet_weights/lstm/decoder_lstm/weights.36-1.27.hdf5')

In [13]:
predictions = decoder.predict()

3313/3313 [==============================] - 6s 2ms/step


In [36]:
predicted_classnames = map_predictions_to_classnames(predictions, true_classes, true_classnames)